<a href="https://colab.research.google.com/github/LauraSchellekens/TM10007_-private_-group/blob/Laura_random_forest/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# toepassen random forest op onze data 

# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git
!pip install sklearn numpy matplotlib


In [0]:
# General packages
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import model_selection

In [7]:
# Data loading functions. 
from adni.load_data import load_data
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
X = data.drop('label', axis=1) # All data without column 'label'
Y = data['label'] # AD en CN moeten alleen nog omgezet worden naar 0 en 1
Y.replace(('AD', 'CN'), (1, 0), inplace=True) # AD en CN omgezet naar 0 en 1


ERROR! Session/line number was not unique in database. History logging moved to new session 61
The number of samples: 855
The number of columns: 268


In [9]:
# Grid search in cross validation setting

# Create a 20 fold stratified CV iterator
cv_20fold = model_selection.StratifiedKFold(n_splits=10)
results = []
best_n_trees = []

# Loop over the folds
for train_index, validation_index in cv_20fold.split(X, Y): #split train data in training and validation data

    # Split the data properly
    X_train = X[train_index]
    y_train = Y[train_index]
    
    X_validation = X[validation_index]
    y_validation = Y[validation_index]

    # Create a grid search to find the optimal k using a gridsearch and 10-fold cross validation
    parameters = {"n_trees": list(range(1, 26, 2))} # range moet gekozen worden 
    rfc = n_estimators.RandomForestClassifier() 
    cv_10fold = model_selection.StratifiedKFold(n_splits=10) # folds moet gekozen worden en stratification
    grid_search = model_selection.GridSearchCV(rfc, parameters, cv=cv_10fold, scoring='roc_auc') #scoring moet gekozen worden
    grid_search.fit(X_train, y_train)

    # Get resulting classifier
    clf = grid_search.best_estimator_
    print(f'Best classifier: k={clf.n_estimators}')
    best_n_trees.append(clf.n_estimators)

    # Test the classifier on the test data
    probabilities = clf.predict_proba(X_validation)
    scores = probabilities[:, 1]
    
    # Get the auc
    auc_validation = metrics.roc_auc_score(y_validation, scores)
    results.append({
        'auc': auc_validation, 
        'k': clf.n_estimators,
        'set': 'validation'
    })

# Create results dataframe and plot it
results = pd.DataFrame(results)
seaborn.boxplot(y='auc', x='set', data=results)

optimal_n = int(np.median(best_n_trees)) # of median
print(f"The optimal N={optimal_n}")

KeyError: ignored

In [0]:
# Use the optimal parameters without any tuning to validate the optimal classifier
clf = RandomForestClassifier(n_estimators=optimal_n)
clf.fit(X, Y)
y_pred = clf.predict(X)
t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))

In [0]:
# scoring classifier   
    auc=metrics.roc_auc_score(Y_r, y_score)
    accuracy=metrics.accuracy_score(Y_r, y_pred)
    F1=metrics.f1_score(Y_r,y_pred)
    precision=metrics.precision_score(Y_r,y_pred)
    recall=metrics.recall_score(Y_r, y_pred)